In [1]:
root_src = 'D:\Dogs vs. Cats'
import os
import shutil
import time
os.chdir(root_src)
import numpy as np
import h5py
import h5py
import numpy as np
from sklearn.utils import shuffle
np.random.seed(2017)

X_train = []
X_test = []

for filename in ["gap_ResNet50.h5", "gap_Xception.h5", "gap_InceptionV3.h5"]:
    with h5py.File(filename, 'r') as h:
        X_train.append(np.array(h['train']))
        X_test.append(np.array(h['test']))
        y_train = np.array(h['label'])

X_train = np.concatenate(X_train, axis=1)
X_test = np.concatenate(X_test, axis=1)

X_train, y_train = shuffle(X_train, y_train)

In [2]:
print(X_train.shape,X_test.shape)

(25000, 6144) (12500, 6144)


In [5]:
from keras.models import *
from keras.layers import *
from keras.callbacks import EarlyStopping
import pandas as pd
from keras.preprocessing.image import *

np.random.seed(2017)
drop_rate_matrix = [0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9]
for i,drop_rate in enumerate(drop_rate_matrix):
    input_tensor = Input(X_train.shape[1:])
    # 可能会有部分特征一致或者类似，所以dropout一些特征，另一方面也可以防止过拟合
    x = Dropout(drop_rate)(input_tensor)
    x = Dense(1, activation='sigmoid')(x)#直接设置全连接
    model = Model(input_tensor, x)

    model.compile(optimizer='adadelta',
                  loss='binary_crossentropy',
                  metrics=['accuracy'])
    
    earlystopping = EarlyStopping(monitor='val_loss', min_delta=0, patience=1, verbose=0, mode='auto')
    model.fit(X_train, y_train, batch_size=128, epochs=10, validation_split=0.2,callbacks=[earlystopping])
    
    y_pred = model.predict(X_test, verbose=1)
    y_pred = y_pred.clip(min=0.005, max=0.995)

    df = pd.read_csv("sample_submission.csv")

    gen = ImageDataGenerator()
    test_generator = gen.flow_from_directory("test", (224, 224), shuffle=False, 
                                             batch_size=16, class_mode=None)
    # 获取文件的序号并减1，然后设置到新预测的csv文件的序号
    for m, fname in enumerate(test_generator.filenames):
        index = int((fname.split('\\')[1]).split('.')[0])
        df.set_value(index-1, 'label', y_pred[m])
    droprate_test_file = 'Best_Model_DropRate_Test_submission'+str(i)+'.csv'
    print('create droprate_test_file',str(i),':',droprate_test_file)
    df.to_csv(droprate_test_file, index=None)

Train on 20000 samples, validate on 5000 samples
Epoch 1/10
20000/20000 [==============================] - 9s - loss: 0.1006 - acc: 0.9791 - val_loss: 0.0336 - val_acc: 0.9908
Epoch 2/10
20000/20000 [==============================] - 4s - loss: 0.0272 - acc: 0.9929 - val_loss: 0.0186 - val_acc: 0.9942
Epoch 3/10
20000/20000 [==============================] - 4s - loss: 0.0195 - acc: 0.9940 - val_loss: 0.0154 - val_acc: 0.9950
Epoch 4/10
20000/20000 [==============================] - 4s - loss: 0.0165 - acc: 0.9948 - val_loss: 0.0130 - val_acc: 0.9954
Epoch 5/10
20000/20000 [==============================] - 4s - loss: 0.0148 - acc: 0.9953 - val_loss: 0.0127 - val_acc: 0.9954
Epoch 6/10
20000/20000 [==============================] - 4s - loss: 0.0131 - acc: 0.9960 - val_loss: 0.0119 - val_acc: 0.9958
Epoch 7/10
20000/20000 [==============================] - 4s - loss: 0.0126 - acc: 0.9960 - val_loss: 0.0114 - val_acc: 0.9958
Epoch 8/10
20000/20000 [==============================] - 4s -